<a href="https://colab.research.google.com/github/SOURABHMISHRA5221/NaturalLanguageToSQL/blob/main/Natural_Language_To_SQL_FOR_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import random



In [2]:
conn = sqlite3.connect('tables.db') # Connecting to the database
cursor = conn.cursor() # Object to run queries

In [29]:
urls = [
    'https://gist.github.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv',
    'https://gist.github.com/kevin336/5ea0e96813aa88871c20d315b5bf445c/raw/d8fcf5c2630ba12dd8802a2cdd5480621b6a0ea6/departments.csv'
]

# Initialize an empty list to store the DataFrames
dfs = []

# Read each file and append the DataFrame to the list
for url in urls:
    df = pd.read_csv(url)
    dfs.append(df)

In [30]:
create_table_queries = []
for df in dfs:
    # Generate a random table name
    table_name = 'table'+str(random.randint(1, 100000))

    # Create a string of column names with types
    columns_with_types = ", ".join([f"{col.replace(' ', '_')} TEXT" for col in df.columns])

    # Create the CREATE TABLE query
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});"
    create_table_queries.append(create_table_query)

    # Execute the CREATE TABLE query
    cursor.execute(create_table_query)
    cursor.fetchall()

    # Insert data into the table
    for index, row in df.iterrows():
        values = ", ".join([f'"{row_item}"' for row_item in row])
        insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns.str.replace(' ', '_'))}) VALUES ({values})"
        cursor.execute(insert_sql)

In [31]:
prompt = "Give me the employee name, his manager name and his department whose salary is greater than 10000"

In [33]:
import requests

# Define the API endpoint URL and API key
url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent'
api_key = 'ENTER_YOUR_OWN_API_KEY'

# Define the request headers and data
headers = {'Content-Type': 'application/json'}
data = {'contents': [{'parts': [{'text': 'Just give me the SQL query, do not write anything else like ```sql ```. My tables are '+ str(create_table_queries) + ". " + prompt}]}]}

# Set the API key in the params
params = {'key': api_key}

# Send a POST request
response = requests.post(url, headers=headers, json=data, params=params)

# Check if the request was successful
if response.status_code == 200:
    text = response.json()['candidates'][0]['content']['parts'][0]['text']
    print(text)
else:
    print('Failed to retrieve data:', response.status_code)

SELECT e.FIRST_NAME || ' ' || e.LAST_NAME AS EmployeeName, m.FIRST_NAME || ' ' || m.LAST_NAME AS ManagerName, d.DEPARTMENT_NAME AS DepartmentName FROM table28936 e JOIN table28936 m ON e.MANAGER_ID = m.EMPLOYEE_ID JOIN table91993 d ON e.DEPARTMENT_ID = d.DEPARTMENT_ID WHERE CAST(REPLACE(e.SALARY, ',', '') AS REAL) > 10000;



In [34]:
cursor.execute(text)
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(column_names)

# Print the results
for row in results:
    print(row)

['EmployeeName', 'ManagerName', 'DepartmentName']
('Michael Hartstein', 'Steven King', 'Marketing')
('Shelley Higgins', 'Neena Kochhar', 'Accounting')
('Neena Kochhar', 'Steven King', 'Executive')
('Lex De Haan', 'Steven King', 'Executive')
('Nancy Greenberg', 'Neena Kochhar', 'Finance')
('Den Raphaely', 'Steven King', 'Purchasing')
